In [2]:
%reload_ext autoreload
%load_ext dotenv 
%dotenv -o -v .env

from DbConnexion import DbConnexion



Db = DbConnexion(required_privileges={'SELECT'})

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
--- Initialisation de la connexion à la BDD ---
sae204now
Connexion à la base de données sae204now réussie !
--- vérification de la BDD ---
ping...
ping réussi
vérification des permissions...
Récupération des privilèges de l'utilisateur...
vérification des permissions réussie
Vérification des tables...
La table history_change existe
La table station_information existe
La table station_status existe


In [3]:
import folium
import mysql.connector
from mysql.connector import errorcode
from colors import Fore, Style
from cursorContext import cursorContext
import pandas as pd


query = """
SELECT i.stationcode, i.name, i.capacity, ST_X(coordonnees_geo) AS lon, ST_Y(coordonnees_geo) AS lat, 
i.nom_arrondissement_communes, s.date, s.is_installed, s.numdocksavailable, s.numbikesavailable, s.mechanical, s.ebike
    FROM station_information i 
    INNER JOIN station_status s ON s.stationcode = i.stationcode 
    WHERE s.date = ( 
      SELECT MAX(date) 
      FROM station_status 
      WHERE stationcode = i.stationcode 
    )
"""
stations = []
with cursorContext(Db.db) as cursor:
    try:
        cursor.execute(query, ())
    except Exception as e:
        print(Fore.RED + "Erreur lors de récupération des données : " + str(e) + Style.RESET_ALL)

    stations = cursor.fetchall()

data = pd.DataFrame(stations, columns=cursor.column_names)
display(data.head())

,stationcode,name,capacity,lon,lat,nom_arrondissement_communes,date,is_installed,numdocksavailable,numbikesavailable,mechanical,ebike
0,10001,René Boulanger - Lancry,62,2.360003,48.868543,Paris,2023-05-15 09:45:04,OUI,50,12,6,6
1,10003,Hauteville - Bonne Nouvelle,17,2.349650,48.870774,Paris,2023-05-15 09:45:04,OUI,2,14,3,11
2,10004,Metz - Faubourg Saint-Denis,48,2.353521,48.870893,Paris,2023-05-15 09:45:04,OUI,22,25,8,17
3,10005,Petites Ecuries - Faubourg Poissonnière,17,2.348389,48.873972,Paris,2023-05-15 09:45:04,OUI,2,14,5,9
4,10006,Petites Ecuries - Faubourg Saint-Denis,29,2.354215,48.872871,Paris,2023-05-15 09:45:04,OUI,7,20,16,4


In [17]:
from folium.features import DivIcon

def generate_icon(perc, text, color):
    return DivIcon(
        html=f"""
        <svg width="50px" height="50px" viewBox="0 0 42 42" className="donut" aria-labelledby="beers-title beers-desc" role="img">
                <circle className="donut-hole" cx="21" cy="21" r="15.91549430918954" fill="white" role="presentation"></circle>
                <circle className="donut-ring" cx="21" cy="21" r="15.91549430918954" fill="transparent" stroke="#d2d3d4" strokeWidth="3" role="presentation"></circle>
                <circle className="donut-segment" cx="21" cy="21" r="15.91549430918954" fill="transparent" stroke="{color}" strokeWidth="3" strokeDasharray="{perc} {100 - perc}" strokeDashoffset="25" aria-labelledby="donut-segment-1-title donut-segment-1-desc">
                </circle>
                <g className="chart-text">
                    <text className="chart-number" x="50%" y="52%" dominantBaseline="middle" textAnchor="middle">
                        {text}
                    </text>
                </g>
            </svg>
        """,
        icon_size=(50, 50),
        popup_anchor=(0, -30)
    )

In [18]:
import folium
from folium.plugins import MarkerCluster

print(len(data))

# filtrer donnée seulement paris
filter_data = data.loc[data['nom_arrondissement_communes'].str.lower() == 'paris']


m = folium.Map(location=[48.856, 2.352], zoom_start=12)
	
marker_cluster = MarkerCluster().add_to(m)


for index, row in filter_data.iterrows():
    pourc = round((row['numbikesavailable'] / row['capacity']) * 100) if row['capacity'] > 0 else 0;
    folium.Marker([row['lat'], row['lon']], popup=row['name'], icon=generate_icon(perc=pourc, text=row['numbikesavailable'], color="hsl(130, 100%, 40%)")).add_to(marker_cluster)

m

1466
